[Covid-19 Dashboard](https://github.com/yasmin-lm/Covid-19-dashboard) (C) Yasmin Mohammadi, 2023 ([y.mohammadi@se23.qmul.ac.uk](mailto:y.mohammadi@se23.qmul.ac.uk) - [web](http://www.uk.linkedin.com/in/yasmin-mohammadi-63470b143?trk=public_profile_browsemap)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# Covid-19 Dashboard

This Covid Dashboard, is displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. 

The purpose of this dashboard, is to follow trends in Covid testing undertaken by Public Health England during, and following the COVID-19 pandemic. This data focuses on the number of new cases reported on a given date and the cumulative tests in the same time period. 


In [99]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [100]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load of data

The data for this dashboard is accessed through a web-based Application Programming Interface (API). It exchanges information with the Public Health England Coronavirus dashboard in JSON format, based on the *http* protocol. 

For this dashboard, data relating to the region: *England*, as well as data relating to *cumulative PCR tests, number of new cases* and *date of testing* was retrieved and packaged into tables and interactive graphs.

In [101]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open ("pcrtests.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

## Wrangle the data

The dashboard wrangles the JSON data retrieved from the API to create relevant tables and graphs such as Table 1 below. From this, we can identify trends in case numbers through the pandemic and following the pandemic. 

Table 1. Table showing the number of PCR cases tested positive and the total number cumulative tests on a given date. The table only shows the first 5 and last five entries taken from the API.

In [102]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    pcrdatalist=jsondata['data']

    pcrdates=[dictionary['date'] for dictionary in pcrdatalist ]
    pcrdates.sort()

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
        
    startdate=parse_date(pcrdates[0])
    enddate=parse_date(pcrdates[-1])
        
    index=pd.date_range(startdate, enddate, freq='D')
    pcrtestsdf=pd.DataFrame(index=index, columns=['cases', 'tests'])

    for entry in pcrdatalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'tests']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(pcrtestsdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                pcrtestsdf.loc[date, column]=value
                
                # fill in any remaining "holes" due to missing dates
    pcrtestsdf.fillna(0.0, inplace=True)

    return pcrtestsdf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
pcrtestsdf=wrangle_data(jsondata) # df is the dataframe for plotting
print (pcrtestsdf)

            cases        tests
2020-01-03    0.0        287.0
2020-01-04    0.0        422.0
2020-01-05    0.0        735.0
2020-01-06    0.0        793.0
2020-01-07    0.0        930.0
...           ...          ...
2023-10-14  363.0  223033424.0
2023-10-15  321.0  223037650.0
2023-10-16  472.0  223042266.0
2023-10-17  320.0  223043396.0
2023-10-18  120.0          0.0

[1385 rows x 2 columns]


## Download current data

In order to view the most current data for this dataset, please click the button below to refresh this page. This will retrieve the most recent information from the Public Health England Coronavirus database. 

In [112]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
        
    filters = [
    'areaType=nation',
    'areaName=England'
    ]
    
    
    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "cases": "newCasesPCROnlyBySpecimenDate",
        "tests": "cumPCRTestsBySpecimenDate"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    
    pcrtests=api.get_json()
            
    return pcrtests # return data read from the API


In [104]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global pcrtestsdf
    pcrtestsdf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Referesh data', # you may want to change this...
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Referesh data', icon='download', style=ButtonStyle(), tooltip='Click to download current P…

## Graphs and Analysis

Graph 1 below, shows the number of new cases tested by PCR and the total cumulative PCR tests for a given date.

Use the interactive widgets to change the way the graph is viewed. 

In [105]:
def plot_pcrtests():
    pcrtestsdf.plot()
    pcrtestsdf.plot(logy=True)
    plt.show() # important! update won't work properly without this
    
tests=wdg.SelectMultiple(
    options=['cases', 'tests'],
    value=['cases', 'tests'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([tests, scale])

def pcrtests_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        pcrtestsdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options['linear']:
        other=scale.options['log']
    else:
        other=scale.options['linear']
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back
    
    
graph=wdg.interactive_output(pcrtests_graph, {'gcols': tests, 'gscale': scale})
    
display(controls, graph)

Output()

Graph 1. PCR tests and positive cases tested by Public Health England from January 2020, to date.

**Author and Copyright Notice** 

Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Yasmin Mohammadi,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).